In [ ]:
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd

In [ ]:
cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(cluster)
client

In [ ]:
df = dd.read_parquet('/project/shared/data/gaia_dr2/gaia_source_with_rv.parquet/*', columns=['l', 'b', 'radial_velocity'], engine='pyarrow')

In [ ]:
def rot_l(x):
    l = x['l']
    if l > 180.:
        x['l'] = l-360.
    return x

In [ ]:
df = df.apply(rot_l, axis=1)

In [ ]:
%%time
df = df.compute()

In [ ]:
%%time
len(df)

In [ ]:
import holoviews as hv
import holoviews.operation.datashader as hd
import datashader as ds
hd.shade.cmap=["darkblue", "red"]
hv.extension("bokeh", "matplotlib")

In [ ]:
points = hv.Points(df, kdims=['l', 'b'], vdims=['radial_velocity'])

In [ ]:
%%time
%%opts RGB [width=1000, height=500, colorbar=True]
hd.datashade(points, aggregator=ds.mean('radial_velocity'))

In [ ]:
cluster.close()